## Import libraries

In [2]:
import pandas as pd


## Read in excel file to pandas

In [7]:
rule_df = pd.read_excel(
    "../data/raw/RiskClassification_Data_Endpoints_V2_Shared.xlsx", sheet_name="RiskRules")

# drop the first coulmn
rule_df = rule_df.iloc[:, 1:]
rule_df


,authentication,security_test_category,security_test_result,server_location,hosting_isp,PII,FII,Risk_Label
0,No Authentication,Injections,Fail,Anywhere,Anyone,NaN,NaN,Imminent
1,No Authentication,Injections,Fail,Anywhere,Anyone,NaN,NaN,Imminent
2,No Authentication,Injections,Fail,Anywhere,Anyone,NaN,NaN,Imminent
3,No Authentication,Injections,Fail,Anywhere,Anyone,NaN,NaN,Imminent
4,No Authentication,Injections,Fail,Anywhere,Anyone,NaN,NaN,Imminent
...,...,...,...,...,...,...,...,...
70,Some Authentication,All Tests Performed/Available,Pass,Amaricas,Anyone,Yes,Yes,Low
71,Some Authentication,All Tests Performed/Available,Pass,Amaricas,Anyone,Yes,Yes,Low
72,Some Authentication,All Tests Performed/Available,Pass,Amaricas,Anyone,Yes,Yes,Low
73,Some Authentication,All Tests Performed/Available,Pass,Amaricas,Anyone,Yes,Yes,Low


In [17]:
# find the unique values in column "authentication"
auth_col = rule_df["authentication"].unique()
print(f'Unique values in column "authentication": {auth_col}')

# find the unique values in column "security_test_category"
sec_test_col = rule_df["security_test_category"].unique()
print(f'Unique values in column "security_test_category": {sec_test_col}')

# find the unique values in column "security_test_result"
sec_test_res_col = rule_df["security_test_result"].unique()
print(f'Unique values in column "security_test_result": {sec_test_res_col}')

# find the unique values in column "server_location"
server_loc_col = rule_df["server_location"].unique()
print(f'Unique values in column "server_location": {server_loc_col}')

# find the unique values in column "hosting_isp"
host_isp_col = rule_df["hosting_isp"].unique()
print(f'Unique values in column "hosting_isp": {host_isp_col}')

# find the unique values in column "PII"
pii_col = rule_df["PII"].unique()
print(f'Unique values in column "PII": {pii_col}')

# find the unique values in column "FII"
fii_col = rule_df["FII"].unique()
print(f'Unique values in column "FII": {fii_col}')

# find the unique values in column "Risk_Label"
risk_label_col = rule_df["Risk_Label"].unique()
print(f'Unique values in column "Risk_Label": {risk_label_col}')


Unique values in column "authentication": ['No Authentication' 'Some Authentication']
Unique values in column "security_test_category": ['Injections' 'Broken Authentication' 'No Test Performed/Available'
 'All Tests Performed/Available' 'Buffer Overflow' 'XML External Enteties'
 'Cross-Site Scripting']
Unique values in column "security_test_result": ['Fail' nan 'Pass']
Unique values in column "server_location": ['Anywhere' 'Russia' 'China' 'Others' 'West Europe' 'Amaricas']
Unique values in column "hosting_isp": ['Anyone']
Unique values in column "PII": [nan 'Yes' 'No']
Unique values in column "FII": [nan 'Yes' 'No']
Unique values in column "Risk_Label": ['Imminent' 'High' 'Medium' 'Low']


In [ ]:
# write the rule to classify the Risk_Label based on rule_df
def classify_risk(row):
    """
    Check if the row is the same with any row in rule_df
    """
    # loop through each row in rule_df
    for index, row_rule in rule_df.iterrows():
        # check if the row is the same with any row in rule_df
        if row.equals(row_rule):
            # return the value in column "Risk_Label"
            return row_rule["Risk_Label"]
    return "No Match"
    
    